In [111]:
import pandas as pd
import locale

In [116]:
data = pd.read_csv('/Users/jihyunlee/KoreaUniv/21FW/IntroDS/00.프로젝트/movie_list_add_ratio_1129.csv', encoding='euckr')
data.columns = ['Unnamed: 0','Unnamed: 1', 'MOVIE_NM', 'MNG_NM', 'MNG_CD', 'MAKR_NM',
       'IMPORT_CMPNY_NM', 'DISTB_CMPNY_NM', 'OPEN_DE', 'MOVIE_TY_NM',
       'MOVIE_STLE_NM', 'NLTY_NM', 'WNTY_SCREEN_CO', 'WNTY_SELNG_AM',
       'WNTY_AUDE_CO', 'SU_SELNG_AM', 'SU_AUDE_CO', 'GENRE_NM', 'GRAD_NM',
       'MOVIE_SE','lotte_3_days', 'lotte_7_days',
       'lotte_15_days', 'cgv_3_days', 'cgv_7_days', 'cgv_15_days',
       'jcon_3_days', 'jcon_7_days', 'jcon_15_days', 'naver_cd']


a = data[['naver_cd','MOVIE_NM','DISTB_CMPNY_NM','OPEN_DE','MOVIE_STLE_NM','NLTY_NM','WNTY_SCREEN_CO','GENRE_NM','GRAD_NM','MOVIE_SE']]
b = data[data.columns[20:29]]

df = pd.concat([a,b],axis=1)

df.columns = ['movie_code','movie_name','distribution_company','open_date','movie_style','national','screen_number',
              'genre','grade','movie_type','lotte_3_days','lotte_7_days','lotte_15_days',
              'cgv_3_days','cgv7_days','cgv_15_days','jcon_3_days','jcon_7_days','jcon_15_days']

#배급사명처리
tmp = []
for i in range(len(df['distribution_company'])):
    tmp.append( df['distribution_company'][i].split(',')[0] )
df['distribution_company'] = tmp

#one-hot encoding
df = pd.get_dummies(df, columns = ['distribution_company','movie_style','national','genre','grade','movie_type'])

#text review join
text = pd.read_csv('/Users/jihyunlee/KoreaUniv/21FW/IntroDS/00.프로젝트/text.csv')

grouped_score = text['score'].groupby(text['movie_code'])
score_mean = grouped_score.mean()
score_std = grouped_score.std()

score_cnt = grouped_score.count() >= 200
score_cnt = score_cnt*1

grouped_sensitive_score = text['sensitive_score'].groupby(text['movie_code'])
sensitive_score_mean = grouped_score.mean()
sensitive_score_std = grouped_score.std()

df = pd.merge(df, score_cnt, left_on='movie_code', right_on='movie_code', how='left')
df.rename( {'score':'review_over_200'}, axis='columns', inplace=True)

df = pd.merge(df, score_mean, left_on='movie_code', right_on='movie_code', how='left')
df.rename( {'score':'score_mean'}, axis='columns', inplace=True)

df = pd.merge(df, score_std, left_on='movie_code', right_on='movie_code', how='left')
df.rename( {'score':'score_std'}, axis='columns', inplace=True)

df = pd.merge(df, sensitive_score_mean, left_on='movie_code', right_on='movie_code', how='left')
df.rename( {'score':'sensitive_score_mean'}, axis='columns', inplace=True)

df = pd.merge(df, sensitive_score_std, left_on='movie_code', right_on='movie_code', how='left')
df.rename( {'score':'sensitive_score_std'}, axis='columns', inplace=True)

col1 = df.columns[:4].to_list()
col2 = df.columns[4:13].to_list()
col3 = df.columns[13:].to_list()

new_col = col1+col3+col2
df = df[new_col]

#결측치제거
df = df[ df['movie_code'].notna() ]
df = df.fillna(0)

df.to_csv('(final)movie_data_set.csv',header=True,index=False)


In [117]:
#전국 매출금액과 등락률의 상관관계
exploror = b.copy()
exploror['WNTY_SELNG_AM'] = data['WNTY_SELNG_AM']
exploror.WNTY_SELNG_AM = exploror.WNTY_SELNG_AM.apply(lambda x : x.replace(',',''))
exploror = exploror.apply(pd.to_numeric)
exploror.corr()

,lotte_3_days,lotte_7_days,lotte_15_days,cgv_3_days,cgv_7_days,cgv_15_days,jcon_3_days,jcon_7_days,jcon_15_days,WNTY_SELNG_AM
lotte_3_days,1.000000,0.752213,0.507150,0.384215,0.230851,0.219246,0.164338,0.265744,0.008430,0.033406
lotte_7_days,0.752213,1.000000,0.633837,0.295031,0.279396,0.234289,0.074309,0.192374,-0.118626,0.019432
lotte_15_days,0.507150,0.633837,1.000000,0.253426,0.270250,0.410399,0.062651,0.230667,-0.277021,0.003909
cgv_3_days,0.384215,0.295031,0.253426,1.000000,0.742613,0.490684,0.283312,0.340895,-0.107890,0.005621
cgv_7_days,0.230851,0.279396,0.270250,0.742613,1.000000,0.720621,0.340525,0.460714,-0.217865,0.006028
cgv_15_days,0.219246,0.234289,0.410399,0.490684,0.720621,1.000000,0.168213,0.329930,-0.187861,0.010183
jcon_3_days,0.164338,0.074309,0.062651,0.283312,0.340525,0.168213,1.000000,0.720330,0.017471,-0.069818
jcon_7_days,0.265744,0.192374,0.230667,0.340895,0.460714,0.329930,0.720330,1.000000,0.032760,-0.049014
jcon_15_days,0.008430,-0.118626,-0.277021,-0.107890,-0.217865,-0.187861,0.017471,0.032760,1.000000,-0.032367
WNTY_SELNG_AM,0.033406,0.019432,0.003909,0.005621,0.006028,0.010183,-0.069818,-0.049014,-0.032367,1.000000
